In [4]:
import re
from collections import Counter

# Import the corpus and get the count of each word
WORDS = open("corpus.txt").read()
WORDS = re.findall('[a-zA-Z0-9]+', WORDS.lower())
WORDS = Counter(WORDS)

In [5]:
# The following functions return the set of words with edit distance at one and two respectively.
# One edit incluldes Deletion, Transposition, Replacement, and Inserts.

def edits_at_one_distance(word):
    alphabet = "abcdefghijklmnopqrstuvwxyz"

    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    # print(deletes)

    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    # print(transposes)

    replaces = [L + c + R[1:] for L, R in splits if R for c in alphabet]
    # print(replaces)

    inserts = [L + c + R for L, R in splits for c in alphabet]
    # print(inserts)

    return set(deletes + transposes + replaces + inserts)

def edits_at_two_distance(word):
    return (e2 for e1 in edits_at_one_distance(word) for e2 in edits_at_one_distance(e1))

In [8]:
# Reduce the length of word. A same alphabet does not occur more than twice in a word in English.
def reduce_length(word):
#     print(re.sub(r'(.)\1{2,}', r'\1\1', word))
#     pattern = re.compile(r'(.)\1{2,}')
#     print(pattern.sub(r'\1\1', 'a', word))
    return re.sub(r'(.)\1{2,}', r'\1\1', word)

# This is the utility function to return the probability of a given word in the corpus.
def probability_of_word(word, N=sum(WORDS.values())):
    return WORDS[word] / N

# This function returns the correction of a given incorrect word.
def correction(word):
    word = reduce_length(word)
    return max(candidates(word), key=probability_of_word)

# This function returns all the possible candidates of a given words
def candidates(word):
    return (known([word]) or known(edits_at_one_distance(word)) or known(edits_at_two_distance(word)) or [word])

# This function returns all the known words based on the corpus among the can
def known(words):
    return set(word for word in words if word in WORDS)

In [15]:
print(correction("pattern"))
print(correction("amaziiingggggggg"))
print(correction("speling"))
print(correction("currected"))
print(correction("currect"))

pattern
amazing
spelling
corrected
current
